Realizaremos el primer proceso de limpieza, teniendo en cuenta los siguientes puntos:

A.- Valores nulos

B.- Valores duplicados

C.- datos faltantes

D.- datos no necesarios

# Análisis exploratorio

El análisis exploratorio nos dará información estructural de los dataframes (tamaño, orden, etc) para poder realizar el primer proceso de limpieza, en donde queden unicamente datos relevantes, para que posteriormente se haga una transformación (reducción)

In [176]:
import pandas as pd
#Importamos 5 cotizaciones para realizar la parte de limpieza.

cot1 = pd.read_csv('C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones (caratula)/Capa1Extract/2023/Enero/CTQ-787 ELECTROSERVICIOS (DESCARGA Y ARRASTRE DE MAQUINA).csv')
cot2 = pd.read_csv('C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones (caratula)/Capa1Extract/2023/Febrero/CTQ-844 BUSSMANN (SERVICIO DE SUMINISTRO DE RUEDA).csv')
cot3 = pd.read_csv('C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones (caratula)/Capa1Extract/2023/Julio/CTQ-952 ZF (EMBALAJE LINEA EP5).csv')
cot4 = pd.read_csv('C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones (caratula)/Capa1Extract/2023/Septiembre/CTQ-995 PIZZA HUT (SERVICIO DE FABRICACIÓN DE PUERTA PARA ÁREA DEL SITE).csv')
cot5 = pd.read_csv('C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones (caratula)/Capa1Extract/2023/Noviembre/CTQ-1042 VALEO (SUMINISTRO DE VIDRIO TEMPLADO).csv')

Los dataframes provienen de diccionarios, es decir, las columnas: range, majorDimensio, provienen de la funcion de extracción de Google Sheets.

La columna importantes es unicamente "value", extraeremos esa información, quitando las entradas que representan los terminos y condiciones



In [179]:
import io
 #Elimanos espacios vacios y la informacion despues de 'Terminos y condiciones
cotizacion1 = pd.DataFrame(cot1['values'])


j=0

while j<len(cotizacion1):
    if '[]' in cotizacion1.iloc[j,0]:
        print(f"el registro {j} es nulo")
        cotizacion1 = cotizacion1.drop(index = cotizacion1.index[j])
        print(f"dimension interaccion {n}")
        print(f"dimension real: {len(cotizacion1)}")
    else:
        print(f"registro {j} no nulo")
        print(f"dimension real: {len(cotizacion1)}")
        j+=1
        
        

el registro 0 es nulo
dimension interaccion 19
dimension real: 21
el registro 0 es nulo
dimension interaccion 19
dimension real: 20
el registro 0 es nulo
dimension interaccion 19
dimension real: 19
registro 0 no nulo
dimension real: 19
registro 1 no nulo
dimension real: 19
registro 2 no nulo
dimension real: 19
registro 3 no nulo
dimension real: 19
registro 4 no nulo
dimension real: 19
registro 5 no nulo
dimension real: 19
el registro 6 es nulo
dimension interaccion 19
dimension real: 18
registro 6 no nulo
dimension real: 18
registro 7 no nulo
dimension real: 18
registro 8 no nulo
dimension real: 18
registro 9 no nulo
dimension real: 18
registro 10 no nulo
dimension real: 18
registro 11 no nulo
dimension real: 18
registro 12 no nulo
dimension real: 18
registro 13 no nulo
dimension real: 18
registro 14 no nulo
dimension real: 18
registro 15 no nulo
dimension real: 18
registro 16 no nulo
dimension real: 18
registro 17 no nulo
dimension real: 18


In [184]:
#Quitamos para la cotizacion2, toda la informacion despues de terminos y condiciones:
cotizacion2 = cot2[['values']]

n= 0
while n< len(cotizacion2):
    if 'TÉRMINOS' in cotizacion2.iloc[n,0]:
        print(f"A partir del registro {n} tiene los terminos y condiciones")
        cotizacion2 = cotizacion2.drop(index = cotizacion2.index[n:])
        break
    else:
        print(f"La información de este registro {n} es importante")
        n+=1

La información de este registro 0 es importante
La información de este registro 1 es importante
La información de este registro 2 es importante
La información de este registro 3 es importante
La información de este registro 4 es importante
La información de este registro 5 es importante
La información de este registro 6 es importante
La información de este registro 7 es importante
La información de este registro 8 es importante
La información de este registro 9 es importante
La información de este registro 10 es importante
La información de este registro 11 es importante
La información de este registro 12 es importante
La información de este registro 13 es importante
La información de este registro 14 es importante
A partir del registro 15 tiene los terminos y condiciones


In [ ]:
val1 = val1[~val1.iloc[:, 0].str.contains(r'^\[\]$', na=False)]

# Resetear los índices si es necesario
val1.reset_index(drop=True, inplace=True)

print(val1)

In [192]:

cotizacion3 = cot3[['values']]
i = 0

while i<len(cotizacion3):
    
    if '[]' in cotizacion3.iloc[i,0]:
        print(f"Espacio vacio en el registro {i}")
        cotizacion3 = cotizacion3.drop(index = cotizacion3.index[i])
    
    elif cotizacion3.iloc[i,0] == 'TÉRMINOS': #TÉRMINOS' or 'TERMINOS' in cotizacion3.iloc[i,0]:
        print(f"A partir del registro {i} esta la información de terminos y condiciones.")
        cotizacion3 = cotizacion3.drop(index = cotizacion3.index[i:])
        break
    
    else:
        print("El valor del registro: {i} es importante")
        i+=1    
        
    

Espacio vacio en el registro 0
Espacio vacio en el registro 0
Espacio vacio en el registro 0
Espacio vacio en el registro 0
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
Espacio vacio en el registro 6
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i} es importante
El valor del registro: {i

In [193]:
cotizacion4 = pd.DataFrame(cot1['values'])


j=0

while j<len(cotizacion1):
    if '[]' in cotizacion1.iloc[j,0]:
        print(f"el registro {j} es nulo")
        cotizacion1 = cotizacion1.drop(index = cotizacion1.index[j])
        print(f"dimension interaccion {n}")
        print(f"dimension real: {len(cotizacion1)}")
    else:
        print(f"registro {j} no nulo")
        print(f"dimension real: {len(cotizacion1)}")
        j+=1